In [29]:
# Данный код представляет собой пример использования модели случайного леса (RandomForestClassifier) 
# для решения задачи классификации выживаемости пассажиров титаника.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Embarked']

# Преобразование категориальных признаков в числовые
X_tr = pd.get_dummies(train_data[features])
y_tr = train_data['Survived']
X_test = pd.get_dummies(test_data[features])
PassengerId = train_data['PassengerId']

# Создаём экземпляр класса с заданной стратегией SimpleImputer
imputer = SimpleImputer(strategy='mean')
# Далее применяем метод fit_transform() к матрице X для заполнения пропущенных значений средним значением и преобразует результат в объект DataFrame
X_tr = pd.DataFrame(imputer.fit_transform(X_tr), columns = X_tr.columns)
X_test = pd.DataFrame(imputer.fit_transform(X_test), columns = X_test.columns)

# Разделение данных на тренировочную и валидационную выборки
X_train, X_val, y_train, y_val = train_test_split(X_tr, y_tr, test_size=0.2, random_state=1)

# Масштабирование данных
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# Обучение модели с помощью RandomForestClassifier, 100 деревьев
model = RandomForestClassifier(n_estimators=100, random_state=1)
# Далее идёт обучение модели
model.fit(X_train_scaled, y_train)

# Оценка модели
y_pred = model.predict(X_val_scaled)
accuracy = accuracy_score(y_val, y_pred)
print('Accuracy:', accuracy)

# Предсказание на тестовых данных: Масштабирование тестового набора данных
X_test_scaled = scaler.transform(X_test)
# Предсказание значений целевой переменной (Survived) для тестового набора данных с помощью метода .predict()
predictions = model.predict(X_test_scaled)
# Затем, предсказанные значения преобразуются в бинарные значения (0 или 1) с помощью выражения (predictions > 0.5).astype(int)
predictions = (predictions > 0.5).astype(int)

output = pd.DataFrame({
    'PassengerId': test_data.PassengerId,
    'Survived': predictions
})

# Создание файла с предсказаниями
output.to_csv('prediction.csv', index=False)

Accuracy: 0.7653631284916201


In [30]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.impute import KNNImputer
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier

train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Embarked']

X_tr = pd.get_dummies(train_data[features])
y_tr = train_data['Survived']
X_test = pd.get_dummies(test_data[features])
PassengerId = train_data['PassengerId']

# Создаём экземпляр класса с заданной стратегией KNNImputer, алгоритм, 
# использующий метод k-ближайших соседей для заполнения пропущенных значений в данных
# n_neighbors - количество ближайших соседей
# weights='distance' - весовое значение
# Опция 'distance' означает, что каждое пропущенное значение будет заполнено с учетом обратного расстояния до каждого из его ближайших соседей
imputer = KNNImputer(n_neighbors=5, weights='distance')
X_tr = pd.DataFrame(imputer.fit_transform(X_tr), columns=X_tr.columns)
X_test = pd.DataFrame(imputer.transform(X_test), columns=X_test.columns)

# Масштабирование данных
scaler = StandardScaler()
X_tr_scaled = scaler.fit_transform(X_tr)
X_test_scaled = scaler.transform(X_test)

# Определение сетки параметров param_grid для поиска оптимальных значений гиперпараметров модели
# В данном случае, исследуется влияние: 
# количество деревьев (n_estimators)
# глубина деревьев (max_depth)
# коэффициент обучения (learning_rate) на производительность модели
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.1, 0.05, 0.01]
}

# Создание объекта модели класса GradientBoostingClassifier
model = GradientBoostingClassifier(random_state=1)
# Создание объекта grid_search класса GridSearchCV для выполнения поиска по сетке значений параметров
# Используется 5-кратная кросс-валидация (cv=5)
grid_search = GridSearchCV(model, param_grid, cv=5)
# Обучение модели на тренировочных данных
grid_search.fit(X_tr_scaled, y_tr)

# Получение наилучшей модели best_model с помощью атрибута best_estimator_ объекта grid_search
best_model = grid_search.best_estimator_
# Вывод наилучших параметров модели best_params с помощью атрибута best_params_
best_params = grid_search.best_params_
print('Best parameters:', best_params)

# Применение метода predict() наилучшей модели best_model к данным X_scaled для получения предсказанных меток и 
# вычисление точности модели на обучающем наборе данных с помощью функции accuracy_score()
y_pred = best_model.predict(X_scaled)
accuracy = accuracy_score(y, y_pred)
print('Training Accuracy:', accuracy)

# Применение метода predict() наилучшей модели best_model к данным X_test_scaled для получения 
# предсказания выживаемости пассажиров на тестовом наборе данных
predictions = best_model.predict(X_test_scaled)
output = pd.DataFrame({
    'PassengerId': test_data.PassengerId,
    'Survived': predictions
})

output.to_csv('prediction.csv', index=False)


Best parameters: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100}
Training Accuracy: 0.8686868686868687
